[TOC]

![figure](img/data_wrangling_packages.png)

# DataFrames.jl

In [61]:
using DataFrames
using Pipe

## 创建

### from Array

`DataFrame(columns::AbstractVecOrMat, names::AbstractVector)`
 
第一个参数接收 Vector 则添加一列；接收 Matrix 则添加多列

In [4]:
using LinearAlgebra
A = diagm(ones(Int, 3))
name = ["a", "b", "c"]
df = DataFrame(A, name)

,a,b,c
,Int64,Int64,Int64
1,1,0,0
2,0,1,0
3,0,0,1


### from Named Vectors

In [5]:
name = ["Sally", "Bob", "Alice", "Hank"]
grade_2020 = [1, 5, 8.5, 4]
df = DataFrame(; name, grade_2020) # 注意，分号一定不能少！
println(df)

4×2 DataFrame
 Row │ name    grade_2020 
     │ String  Float64    
─────┼────────────────────
   1 │ Sally          1.0
   2 │ Bob            5.0
   3 │ Alice          8.5
   4 │ Hank           4.0


### from CSV

In [6]:
using CSV

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1423


#### `CSV.read()`

In [7]:
path = "./data/grades.csv"
df = CSV.read(path, DataFrame) # CSV.jl 将自动推断列的数据类型

,name,grade_2020
,String7,Float64
1,Sally,1.0
2,Bob,5.0
3,Alice,8.5
4,Hank,4.0


In [8]:
my_data = """
    a,b,c,d,e
    Kim,2018-02-03,3,4.0,2018-02-03T10:00
    """
path = "./data/my_data.csv"
write(path, my_data)
df = CSV.read(path, DataFrame)

,a,b,c,d,e
,String3,Date,Int64,Float64,DateTime
1,Kim,2018-02-03,3,4.0,2018-02-03T10:00:00


#### `CSV.write()`

In [9]:
function grades_2020()
    name = ["Sally", "Bob", "Alice", "Hank"]
    grade_2020 = [1, 5, 8.5, 4]
    return DataFrame(;name, grade_2020) 
end

# 定义写函数
function write_grades_csv()
    path = "./data/grades.csv"
    CSV.write(path, grades_2020()) # 运行则返回 path
end

write_grades_csv (generic function with 1 method)

In [10]:
path = write_grades_csv()

"./data/grades.csv"

In [11]:
read(path, String) # 像 .txt 一样读取为字符串

"name,grade_2020\nSally,1.0\nBob,5.0\nAlice,8.5\nHank,4.0\n"

也可以用其他符号作为数据的分隔符

In [12]:
function write_space_separated()
    path = "./data/grades-space-separated.csv"
    CSV.write(path, grades_2020(); delim=' ')
end

write_space_separated (generic function with 1 method)

In [13]:
read(write_space_separated(), String)

"name grade_2020\nSally 1.0\nBob 5.0\nAlice 8.5\nHank 4.0\n"

## 查看基本信息

### `size()`

- `size(df)` 返回 (nrow, ncol)
- `size(df, 1)` 返回 nrow
- `size(df, 2)` 返回 ncol

In [14]:
println(size(df))
println(size(df, 1))
println(size(df, 2))

(1, 5)
1
5


### `first()`/`last()`

`first(df, n)`

查看部分行

### `describe()` 

返回 DataFrame，行为变量名，列为均值、极值、中位数等指标

In [15]:
describe(df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Any,Any
1,a,,Kim,,Kim
2,b,,2018-02-03,2018-02-03,2018-02-03
3,c,3.0,3,3.0,3
4,d,4.0,4.0,4.0,4.0
5,e,,2018-02-03T10:00:00,2018-02-03T10:00:00,2018-02-03T10:00:00


## Convert to Matrix

`Matrix(df)`

In [16]:
Matrix(df)

1×5 Matrix{Any}:
 "Kim"  2018-02-03  3  4.0  2018-02-03T10:00:00

## 选择器

In [71]:
grades_2020()

,name,grade_2020
,String,Float64
1,Sally,1.0
2,Bob,5.0
3,Alice,8.5
4,Hank,4.0


### `.` 提取单列

返回 Vector

In [72]:
grades_2020().name # 只是创建了链接到原始列的 view

4-element Vector{String}:
 "Sally"
 "Bob"
 "Alice"
 "Hank"

### 二维列选择器

选择一列时返回 Vector

多列时返回 DataFrame，而非 Matrix

#### Symbol

In [141]:
grades_2020()[!, [:name, :grade_2020]]

,name,grade_2020
,String,Float64
1,Sally,1.0
2,Bob,5.0
3,Alice,8.5
4,Hank,4.0


#### String

In [85]:
grades_2020()[!, "name"]

4-element Vector{String}:
 "Sally"
 "Bob"
 "Alice"
 "Hank"

#### Int/Vector{Int}/Range/BitVector

数字代表的都是列 index

In [110]:
grades_2020()[!, 1:2]
grades_2020()[!, [1, 2]]
grades_2020()[!, <=(2).([1, 2])]

,name,grade_2020
,String,Float64
1,Sally,1.0
2,Bob,5.0
3,Alice,8.5
4,Hank,4.0


#### `Not()`

In [122]:
grades_2020()[!, Not(:name)]

,grade_2020
,Float64
1,1.0
2,5.0
3,8.5
4,4.0


#### `Between()`

In [151]:
grades_2020()[!, Between(:name, :grade_2020)]

,name,grade_2020
,String,Float64
1,Sally,1.0
2,Bob,5.0
3,Alice,8.5
4,Hank,4.0


#### `:`

表示所有列

In [135]:
grades_2020()[!, :]

,name,grade_2020
,String,Float64
1,Sally,1.0
2,Bob,5.0
3,Alice,8.5
4,Hank,4.0


### 二维行选择器

#### BitVector

In [104]:
grades_2020()[(x -> 2<=x<=3).(1:size(grades_2020(), 1)), :]

,name,grade_2020
,String,Float64
1,Bob,5.0
2,Alice,8.5


#### Int/Vector{Int}/Range

In [105]:
grades_2020()[[2, 3], :]

,name,grade_2020
,String,Float64
1,Bob,5.0
2,Alice,8.5


In [23]:
grades_2020()[1:2, :name]

2-element Vector{String}:
 "Sally"
 "Bob"

当某一列的元素不重复时，可以将 DataFrame 转化为 Dict，通过这一列（主键）获取其他列的值

In [24]:
function lookup_name(name::String)
    df = grades_2020()
    dic = Dict(zip(df.name, df.grade_2020))
    dic[name]
end

lookup_name("Bob")

5.0

In [25]:
df = grades_2020()
collect(zip(df.name, df.grade_2020))

4-element Vector{Tuple{String, Float64}}:
 ("Sally", 1.0)
 ("Bob", 5.0)
 ("Alice", 8.5)
 ("Hank", 4.0)

#### `Not()`

用法同列情形，不赘述

行选择器没有 `Between()` 语法，因为没必要，用 Range 即可

In [153]:
grades_2020()[Not(2), :]

,name,grade_2020
,String,Float64
1,Sally,1.0
2,Alice,8.5
3,Hank,4.0


#### `!`/`:`

都表示所有行，其中 `!` 不像 `:` 拷贝了列，而是创建链接到原始列的 view，因此速度会更快

In [140]:
grades_2020()[!, :]

,name,grade_2020
,String,Float64
1,Sally,1.0
2,Bob,5.0
3,Alice,8.5
4,Hank,4.0


,name,grade_2020
,String,Float64
1,Bob,5.0
2,Alice,8.5
3,Hank,4.0


## 长宽数据转换

### 宽转长 `stack()`

```julia
stack(df::AbstractDataFrame[, measure_vars[, id_vars]];
    variable_name=:variable, 
    value_name=:value, 
    view::Bool=false, 
    variable_eltype::Type=String)
```

第三个参数 id_vars 可选，这一列作为 variable-value 的id

In [212]:
df = DataFrame(a=repeat(1:3, inner=2),
    b=repeat(1:2, inner=3),
    c=repeat(1:1, inner=6),
    d=repeat(1:6, inner=1),
    e=string.('a':'f'))

,a,b,c,d,e
,Int64,Int64,Int64,Int64,String
1,1,1,1,1,a
2,1,1,1,2,b
3,2,1,1,3,c
4,2,2,1,4,d
5,3,2,1,5,e
6,3,2,1,6,f


In [213]:
stack(df, [:c, :d])

,a,b,e,variable,value
,Int64,Int64,String,String,Int64
1,1,1,a,c,1
2,1,1,b,c,1
3,2,1,c,c,1
4,2,2,d,c,1
5,3,2,e,c,1
6,3,2,f,c,1
7,1,1,a,d,1
8,1,1,b,d,2
9,2,1,c,d,3


In [214]:
stack(df, [:c, :d], [:a]) # 丢掉 variable, value, id 以外的变量

,a,variable,value
,Int64,String,Int64
1,1,c,1
2,1,c,1
3,2,c,1
4,2,c,1
5,3,c,1
6,3,c,1
7,1,d,1
8,1,d,2
9,2,d,3


In [215]:
stack(df, Not([:a, :b, :e]))

,a,b,e,variable,value
,Int64,Int64,String,String,Int64
1,1,1,a,c,1
2,1,1,b,c,1
3,2,1,c,c,1
4,2,2,d,c,1
5,3,2,e,c,1
6,3,2,f,c,1
7,1,1,a,d,1
8,1,1,b,d,2
9,2,1,c,d,3


In [217]:
sdf = stack(df, Not([:a, :b, :e]), variable_name=:somemeasure)

,a,b,e,somemeasure,value
,Int64,Int64,String,String,Int64
1,1,1,a,c,1
2,1,1,b,c,1
3,2,1,c,c,1
4,2,2,d,c,1
5,3,2,e,c,1
6,3,2,f,c,1
7,1,1,a,d,1
8,1,1,b,d,2
9,2,1,c,d,3


### 长转宽 `unstack()`

```
unstack(df::AbstractDataFrame, rowkeys, colkey, value; 
    renamecols::Function=identity, allowmissing::Bool=false, allowduplicates::Bool=false, fill=missing)

unstack(df::AbstractDataFrame, colkey, value; 
    renamecols::Function=identity, allowmissing::Bool=false, allowduplicates::Bool=false, fill=missing)

unstack(df::AbstractDataFrame; 
    renamecols::Function=identity, allowmissing::Bool=false, allowduplicates::Bool=false, fill=missing)
```

In [218]:
unstack(sdf, :somemeasure, :value)

,a,b,e,c,d
,Int64,Int64,String,Int64?,Int64?
1,1,1,a,1,1
2,1,1,b,1,2
3,2,1,c,1,3
4,2,2,d,1,4
5,3,2,e,1,5
6,3,2,f,1,6


长转宽有可能出现缺失值，fill 参数决定用什么补充，

In [219]:
df = DataFrame(id=["1", "1", "2"],
    variable=["Var1", "Var2", "Var1"],
    value=[1, 2, 3])

,id,variable,value
,String,String,Int64
1,1,Var1,1
2,1,Var2,2
3,2,Var1,3


In [226]:
df2 = unstack(df, :variable, :value)

,id,Var1,Var2
,String,Int64?,Int64?
1,1,1,2
2,2,3,missing


In [223]:
df2 = unstack(df, :variable, :value, fill = 0)

,id,Var1,Var2
,String,Int64,Int64
1,1,1,2
2,2,3,0


## 筛选行

### 去掉带有缺失值的行

- `dropmissing(df::AbstractDataFrame, cols=:; disallowmissing::Bool=true)`，copy 了一份
- `dropmissing!(df::AbstractDataFrame, cols=:; disallowmissing::Bool=true)`，改变了元数据
    - `dropmissing!(copy(df))`，不会改变元数据

### Filter

`filter(source => f::Function, df)` 筛选 source 列满足条件的行

f 是单体函数，依次作用于 source 列的每个元素上

In [26]:
equals_alice(name::String) = name == "Alice"
filter(:name => equals_alice, grades_2020())

,name,grade_2020
,String,Float64
1,Alice,8.5


In [27]:
filter(:name => n -> n == "Alice", grades_2020()) # 等价写法

,name,grade_2020
,String,Float64
1,Alice,8.5


In [28]:
filter(:name => ==("Alice"), grades_2020()) # 等价写法，缩写了匿名函数

,name,grade_2020
,String,Float64
1,Alice,8.5


### subset

定义这个函数是为了与 DataFrames 中的其他函数保持风格上的一致，第一个参数是 df

`subset(df, source => f)` 中的 f **接收整个 source 列**，返回 AbstractVector（元素为 Bool 或 Bit）  
`subset(df, source => ByRow(f))` 中的 f 为单体函数

In [29]:
subset(grades_2020(), :name => ByRow(==("Alice")))

,name,grade_2020
,String,Float64
1,Alice,8.5


In [52]:
subset(grades_2020(), :name => n -> n .== "Alice") # 不能简写为 .==("Alice")，没有这个语法

,name,grade_2020
,String,Float64
1,Alice,8.5


`subset()` 可以克服 missing 值

In [30]:
function salaries()
    names = ["John", "Hank", "Karen", "Zed"]
    salary = [1_900, 2_800, 2_800, missing]
    DataFrame(; names, salary)
end
salaries()

,names,salary
,String,Int64?
1,John,1900
2,Hank,2800
3,Karen,2800
4,Zed,missing


In [31]:
filter(:salary => >(2_000), salaries()) # 存在 missing 时，filter()失败

LoadError: TypeError: non-boolean (Missing) used in boolean context

In [82]:
subset(salaries(), :salary => ByRow(>(2000)); skipmissing=true) # 加入 skipmissing 参数后，subset 即可成功

,names,salary
,String,Int64?
1,Hank,2800
2,Karen,2800


## 操作列

In [162]:
function responses()
    id = [1, 2]
    q1 = [28, 61]
    q2 = [:us, :fr]
    q3 = ["F", "B"]
    q4 = ["B", "C"]
    q5 = ["A", "E"]
    DataFrame(; id, q1, q2, q3, q4, q5)
end
responses()

,id,q1,q2,q3,q4,q5
,Int64,Int64,Symbol,String,String,String
1,1,28,us,F,B,A
2,2,61,fr,B,C,E


### 选择列

`select(df, args...)`

即使选择单列，也不会退化为矩阵或向量

args...可能的格式：
- Symbol
- String
- Int/Range/Vector{Int}/BitVector
- `Not()`
    - 该函数是 DataFrames 包特有的，接受的参数包括单值、向量、Range
- `Between()`
- `:`，表示前述未包含的所有列，常用于列的重排列
- pair，同时会对取出的列改名

In [165]:
select(responses(), :id, :q1) # Symbol

,id,q1
,Int64,Int64
1,1,28
2,2,61


In [166]:
select(responses(), "id", "q1", "q2") # String

,id,q1,q2
,Int64,Int64,Symbol
1,1,28,us
2,2,61,fr


In [168]:
select(responses(), Not(4:6)) # Not(Range)

,id,q1,q2
,Int64,Int64,Symbol
1,1,28,us
2,2,61,fr


In [169]:
select(responses(), Not([:q4, :q5])) # Not(Vector{Symbol})

,id,q1,q2,q3
,Int64,Int64,Symbol,String
1,1,28,us,F
2,2,61,fr,B


In [182]:
select(responses(), :q5, :q4, Not(:id)) # Not() 的范围是与已经取出的列不同名的所有列

,q5,q4,q1,q2,q3
,String,String,Int64,Symbol,String
1,A,B,28,us,F
2,E,C,61,fr,B


In [183]:
select(responses(), :q5, :) # 此处 : 表示与已经取出的列不同名的所有列

,q5,id,q1,q2,q3,q4
,String,Int64,Int64,Symbol,String,String
1,A,1,28,us,F,B
2,E,2,61,fr,B,C


In [177]:
select(responses(), 1, :q5, :) # Integer

,id,q5,q1,q2,q3,q4
,Int64,String,Int64,Symbol,String,String
1,1,A,28,us,F,B
2,2,E,61,fr,B,C


In [185]:
select(responses(), 1 => "participant", :q1 => "age", :q2, :)

,participant,age,q2,id,q1,q3,q4,q5
,Int64,Int64,Symbol,Int64,Int64,String,String,String
1,1,28,us,1,28,F,B,A
2,2,61,fr,2,61,B,C,E


### 重命名列

- `rename!(df::AbstractDataFrame, vals::AbstractVector{Symbol}; makeunique::Bool=false)`，直接接收新列名向量
- `rename!(df::AbstractDataFrame, vals::AbstractVector{<:AbstractString}; makeunique::Bool=false)`
- `rename!(df::AbstractDataFrame, (from => to)::Pair...)`，接受一堆 pair 的展开
- `rename!(df::AbstractDataFrame, d::AbstractVector{<:Pair})`，接受 pair 的向量
- `rename!(df::AbstractDataFrame, d::AbstractDict)`，接收 Dict
- `rename!(f::Function, df::AbstractDataFrame)`，按一定的规则 f 改名

In [ ]:
df = DataFrame(i=1, x=2, y=3)

In [ ]:
rename!(df, Dict(:i => "A", :x => "X"))

In [ ]:
rename!(df, [:a, :b, :c])

In [ ]:
rename!(df, [:a, :b, :a]) # 会报错，不能两列同名

In [ ]:
rename!(df, [:a, :b, :a], makeunique=true)

In [ ]:
rename!(uppercase, df)

### 排序

```julia
sort!(df::AbstractDataFrame, cols=All();
    alg::Union{Algorithm,Nothing}=nothing,
    lt::Union{Function,AbstractVector{<:Function}}=isless,
    by::Union{Function,AbstractVector{<:Function}}=identity,
    rev::Union{Bool,AbstractVector{Bool}}=false,
    order::Union{Ordering,AbstractVector{<:Ordering}}=Forward)
```

In [43]:
df = DataFrame(x=[3, 1, 2, 1], y=["b", "c", "a", "b"])

,x,y
,Int64,String
1,3,b
2,1,c
3,2,a
4,1,b


In [44]:
sort!(df, :x)

,x,y
,Int64,String
1,1,c
2,1,b
3,2,a
4,3,b


In [45]:
sort!(df, [:x, :y])

,x,y
,Int64,String
1,1,b
2,1,c
3,2,a
4,3,b


In [46]:
sort!(df, [:x, :y], rev=true)

,x,y
,Int64,String
1,3,b
2,2,a
3,1,c
4,1,b


In [47]:
sort!(df, [:x, order(:y, rev=true)])

,x,y
,Int64,String
1,1,c
2,1,b
3,2,a
4,3,b


#### 设定供排序的 levels

按日期排列行

In [ ]:
function wrong_types()
    id = 1:4
    date = ["28−01−2018", "03−04−2019", "01−08−2018", "22−11−2020"]
    age = ["adolescent", "adult", "infant", "adult"]
    DataFrame(; id, date, age)
end
wrong_types()

,id,date,age
,Int64,String,String
1,1,28−01−2018,adolescent
2,2,03−04−2019,adult
3,3,01−08−2018,infant
4,4,22−11−2020,adult


In [ ]:
sort(wrong_types(), :date) # 排序失败，因为格式不标准，Julia只能按照排序字符串的字典顺序对obs排序

,id,date,age
,Int64,String,String
1,3,01−08−2018,infant
2,2,03−04−2019,adult
3,4,22−11−2020,adult
4,1,28−01−2018,adolescent


In [ ]:
using Dates, Pipe
function fix_date_column(df::DataFrame)
    fix_date = d -> Date(d, dateformat"dd−mm−yyyy")
    @pipe df |> 
        transform(_, :date => ByRow(fix_date) => :date)
end
df = fix_date_column(wrong_types())

,id,date,age
,Int64,Date,String
1,1,2018-01-28,adolescent
2,2,2019-04-03,adult
3,3,2018-08-01,infant
4,4,2020-11-22,adult


In [ ]:
sort(df, :date) # 排序成功，是按照日期排的序

,id,date,age
,Int64,Date,String
1,1,2018-01-28,adolescent
2,3,2018-08-01,infant
3,2,2019-04-03,adult
4,4,2020-11-22,adult


按某个字符串列的特定顺序（自定义）排列行

In [ ]:
using CategoricalArrays

In [ ]:
function fix_age_column(df)
    levels = ["infant", "adolescent", "adult"]
    ages = categorical(df[!, :age]; levels, ordered=true)
    df[!, :age] = ages
    df
end
fix_age_column(wrong_types())

,id,date,age
,Int64,String,Cat…
1,1,28−01−2018,adolescent
2,2,03−04−2019,adult
3,3,01−08−2018,infant
4,4,22−11−2020,adult


In [ ]:
df = fix_age_column(wrong_types())
sort(df, :age) # 成功按设定的 levels 排序

,id,date,age
,Int64,String,Cat…
1,3,01−08−2018,infant
2,1,28−01−2018,adolescent
3,2,03−04−2019,adult
4,4,22−11−2020,adult


In [ ]:
function correct_types()
    df = wrong_types()
    df = fix_date_column(df)
    df = fix_age_column(df)
end
correct_types()

,id,date,age
,Int64,Date,Cat…
1,1,2018-01-28,adolescent
2,2,2019-04-03,adult
3,3,2018-08-01,infant
4,4,2020-11-22,adult


### 列的运算

创建新列或覆盖原列

#### `transform()`

`transform(df, source => transformation => target)`，其中 `transformation` 是一个函数，接收整列

可以有 `[:Female_sum, :Male_sum] => ByRow(+) => :Total` 的语法

In [104]:
plus_one(grade) = grade + 1
transform(grades_2020(), :grade_2020 => ByRow(plus_one) => :new_grade_2020)

,name,grade_2020,new_grade_2020
,String,Float64,Float64
1,Sally,1.0,2.0
2,Bob,5.0,6.0
3,Alice,8.5,9.5
4,Hank,4.0,5.0


In [105]:
plus_one_vec(grades) = grades .+ 1
transform(grades_2020(), :grade_2020 => plus_one_vec => :new_grade_2020)

,name,grade_2020,new_grade_2020
,String,Float64,Float64
1,Sally,1.0,2.0
2,Bob,5.0,6.0
3,Alice,8.5,9.5
4,Hank,4.0,5.0


In [110]:
transform(grades_2020(), :grade_2020 => plus_one_vec => :grade_2020) # 覆盖原列

,name,grade_2020
,String,Float64
1,Sally,2.0
2,Bob,6.0
3,Alice,9.5
4,Hank,5.0


In [111]:
transform(grades_2020(), :grade_2020 => plus_one_vec; renamecols=false) # 覆盖原列的另一种写法

,name,grade_2020
,String,Float64
1,Sally,2.0
2,Bob,6.0
3,Alice,9.5
4,Hank,5.0


#### `select()`

或仍然用 `select(df, :, source => transformation => target)`

In [59]:
select(grades_2020(), :, :grade_2020 => plus_one => :grade_2020)

,name,grade_2020
,String,Float64
1,Sally,2.0
2,Bob,6.0
3,Alice,9.5
4,Hank,5.0


#### 赋值修改

In [114]:
df = grades_2020()
df.grade_2020 = plus_one_vec(df.grade_2020)
df

,name,grade_2020
,String,Float64
1,Sally,2.0
2,Bob,6.0
3,Alice,9.5
4,Hank,5.0


## 分组-聚合

`groupby()` + `combine()`

借鉴 R 中的 `groupby()`

In [54]:
using Statistics

In [55]:
group = [:A, :A, :B, :B]
X = 1:4
Y = 5:8
df = DataFrame(; group, X, Y)

,group,X,Y
,Symbol,Int64,Int64
1,A,1,5
2,A,2,6
3,B,3,7
4,B,4,8


In [69]:
gdf = groupby(df, :group)
combine(gdf, [:X, :Y] .=> mean; renamecols=false)

,group,X,Y
,Symbol,Float64,Float64
1,A,1.5,5.5
2,B,3.5,7.5


In [70]:
@pipe df |> 
    groupby(_, :group) |> 
    combine(_, :X => mean => :avgX)

,group,avgX
,Symbol,Float64
1,A,1.5
2,B,3.5


In [121]:
gdf = groupby(df, :group)
rounded_mean(data_col) = round(Int, mean(data_col))
combine(gdf, [:X, :Y] .=> rounded_mean; renamecols=false)

,group,X,Y
,Symbol,Int64,Int64
1,A,2,6
2,B,4,8


## Join

- `innerjoin(A, B; on=[:varA => :varB])`
- `outerjoin(A, B; on=:var)`
- `crossjoin(A, B; makeunique=true)`，行的笛卡尔积，会出现很多 missing 值
- `leftjoin(A, B; on=[:varA => :varB])`
- `rightjoin(A, B; on=:var)`
- `semijoin(A, B; on=:var)`，仅返回存在于左侧 DataFrame 并同时存在于两张 DataFrame 的元素
- `antijoin(A, B; on=:var)`，仅返回存在于左侧 DataFrame 但不存在于右侧 DataFrame 的元素


## 提高性能

- 使用 `!` 版本函数
- 不要拷贝列，即使用 `df.col` 或 `df[!, :col]`，而不是 `df[:, :col]`
- 不要拷贝 csv 数据，即使用 `CSV.read()` 而非 `CSV.File()`
- `CSV.read()` 可以接收数据文件名的向量，并自动执行多线程并行计算，更快地将多个文件读到一个 DataFrame 中
